<a href="https://colab.research.google.com/github/gsilver321/project_gss/blob/main/project1_paper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 1 GSS Paper
### DS 3001: Foundations of Machine Learning
### Gabe Silverstein, Rohan Chowla, Evan Stewart, and Rithwik Raman

## Summary

A one paragraph description of the question, methods, and results (about 350 words)

## Data

One to two pages discussing the data and key variables in the analysis, and any challenges in reading, cleaning, and preparing them for analysis

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

! git clone https://github.com/DS3001/project_gss

Cloning into 'project_gss'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 23 (delta 6), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (23/23), 23.94 MiB | 8.62 MiB/s, done.
Resolving deltas: 100% (6/6), done.


In [60]:
var_list = ['age', 'conrinc', 'educ', 'indus10', 'hrs1', 'prestg10'] # variables to save
output_filename = 'work_data.csv' # name of file output

modes = ['w','a'] # can write and append
phase = 0 # starts in write, switches to append

for k in range(3): # for each data chunk
    url = 'https://github.com/DS3001/project_gss/raw/main/gss_chunk_' + str(1+k) + '.parquet' # get chunk url
    df = pd.read_parquet(url) # download data
    df.loc[:,var_list].to_csv(output_filename, # output file for chunk saving
                              mode=modes[phase], # write vs append
                              header=var_list, # variable names
                              index=False) # no row index saved
    phase = 1 # switch to append

In [61]:
df = df.loc[:, var_list] # only work with variables used in analysis
print('----------- Pre-cleaning null counts')
for v in var_list:
  print(v, sum(df[v].isnull()))

print('\n', df.head(), '\n')

for v in var_list: # loop through each var
  df[v] = df[v].fillna(np.nanmedian(df[v])) # fill nan with median for that column. Using median omits outliers

print('----------- Post-cleaning null counts')
for v in var_list:
  print(v, sum(df[v].isnull()))

print('\n', df.head())

----------- Pre-cleaning null counts
age 591
conrinc 10025
educ 118
indus10 1185
hrs1 10418
prestg10 1129

     age  conrinc  educ  indus10  hrs1  prestg10
0  48.0  59542.0  12.0   9370.0  40.0      49.0
1  25.0   7939.0  16.0   8470.0   NaN      41.0
2  42.0  59542.0  16.0   9490.0  35.0      44.0
3  24.0  33079.0  14.0   9370.0  50.0      35.0
4  24.0  33079.0  16.0   8090.0  40.0      60.0 

----------- Post-cleaning null counts
age 0
conrinc 0
educ 0
indus10 0
hrs1 0
prestg10 0

     age  conrinc  educ  indus10  hrs1  prestg10
0  48.0  59542.0  12.0   9370.0  40.0      49.0
1  25.0   7939.0  16.0   8470.0  40.0      41.0
2  42.0  59542.0  16.0   9490.0  35.0      44.0
3  24.0  33079.0  14.0   9370.0  50.0      35.0
4  24.0  33079.0  16.0   8090.0  40.0      60.0


In [45]:
df

,age,conrinc,educ,indus10,hrs1,prestg10
0,48.0,59542.0,12.0,9370.0,40.0,49.0
1,25.0,7939.0,16.0,8470.0,40.0,41.0
2,42.0,59542.0,16.0,9490.0,35.0,44.0
3,24.0,33079.0,14.0,9370.0,50.0,35.0
4,24.0,33079.0,16.0,8090.0,40.0,60.0
...,...,...,...,...,...,...
24125,22.0,NaN,12.0,8660.0,48.0,30.0
24126,29.0,45360.0,19.0,8190.0,50.0,61.0
24127,32.0,55440.0,15.0,8190.0,38.0,62.0
24128,49.0,45360.0,17.0,7860.0,40.0,61.0


## Results

Two to five pages providing visualizations, statistics, and a discussion of your findings. If you have a lot of plots or tables, that’s OK, but try to focus on a few key pieces of evidence rather than doing every single pairwise comparison of some set of variables

## Conclusion

One to two pages summarizing the project, defending it from criticism, and suggesting additional work that was outside the scope of the project